## HERE BEST

with filtering:
seed = 1337
epoch = 300
adam lr = 0.001
worst rate: 0.3688 (epoch 278)
best rate: 0.8031 (epoch 15)
loss: 0.6332 (epoch: 183, step: 49) -- 0.6332183057069778

---

with filtering:
seed = 1337
epoch = 300
adam lr = 0.0005
worst rate: 0.3625 (epoch 165)
best rate: 0.8062 (epoch 43)
loss: 0.6361 (epoch: 146, step: 49) -- 0.636100463271141

---

with filtering:
seed = 1337
epoch = 300
adam lr = 0.0001
worst rate: 0.5969 (epoch 9)
**best rate: 0.8125** (epoch 267)
loss: 0.6688 (epoch: 262, step: 149) -- 0.6687614250183106

---

without filtering:
seed = 1337
epoch = 100
adam lr = 0.001
worst rate: 0.4875 (epoch 23)
best rate: 0.7125 (epoch 5)
loss: 0.5584 (epoch: 49, step: 299) -- 0.5583532571792602


In [2]:
import copy

import torch
import pickle

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from ds.drivers.datasets import DriversDataset, all_data

In [3]:
seed = 1337
np.random.seed(seed)
torch.random.manual_seed(seed)

In [4]:
ds_all_train = DriversDataset(
    all_data, ds_type="train", data_field="working_data_filtered", step=5
)
ds_all_test = DriversDataset(
    all_data, ds_type="test", data_field="working_data_filtered", step=5
)

In [5]:
base_path = "/home/dmo/Documents/human_func_state"
epoch_count = 300
optim_lr = 0.0001
betas = "default"  # (0.85, 0.995) # Adam only
train_batch_size = 8
test_batch_size = 1
device = "cuda"

In [6]:
dl_train = DataLoader(
    ds_all_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [7]:
dl_test = DataLoader(
    ds_all_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [8]:
class ConvX(nn.Module):
    def __init__(
        self, in_planes, out_planes, kernel=3, stride=1, padding=None
    ):
        super(ConvX, self).__init__()
        padding = kernel // 2 if padding is None else padding
        self.conv = nn.Conv1d(
            in_planes,
            out_planes,
            kernel_size=kernel,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [9]:
class Net6Conv2MaxPool(nn.Module):
    def __init__(self):
        super(Net6Conv2MaxPool, self).__init__()
        self.seq = torch.nn.Sequential(
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            nn.MaxPool1d(2),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            nn.MaxPool1d(2),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, stride=2),
            nn.Conv1d(1, 1, kernel_size=2),
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        return self.softmax(torch.flatten(self.seq(x), 1))

In [10]:
net = Net6Conv2MaxPool().to(device=device)
best_model = Net6Conv2MaxPool().to(device=device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(
#     net.parameters(),
#     lr=optim_lr,
#     betas=betas if isinstance(betas, tuple) else (0.9, 0.999)
# )
optimizer = optim.ASGD(net.parameters())

In [12]:
net(torch.rand((2, 1, 30)).to(device=device))

tensor([[0.7345, 0.2655],
        [0.3409, 0.6591]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [10]:
write_path = (
    f"{base_path}/models_dumps/drivers/"
    f"{net.__class__.__name__}"
    f"_{optimizer.__class__.__name__}"
    f"_lr_{optim_lr}"
    f"_betas_{'_'.join(map(str, betas)) if isinstance(betas, tuple) else betas}"
)
writer = SummaryWriter(log_dir=write_path)

In [11]:
def val(model):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl_test):
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            outputs = model(inputs).cpu()
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [12]:
def train(
    model,
    epoch_count=50,
    print_step=50,
    start_epoch=0,
    min_loss=(np.inf, 0),
    best_rate=(0.0, 0),
    worst_rate=(1.0, 0),
):
    # min_loss = _min_loss
    # best_rate = _best_rate
    # worst_rate = _worst_rate
    len_test = len(dl_test) * dl_test.batch_size
    for epoch in range(
        start_epoch, epoch_count
    ):  # loop over the dataset multiple times
        epoch_loss = 0
        running_loss = 0.0
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.cpu(), labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step - 1:
                mean_loss = running_loss / print_step
                if mean_loss < min_loss[0]:
                    min_loss = (mean_loss, (epoch, i))
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {mean_loss:.3f}")
                epoch_loss += running_loss
                running_loss = 0.0
        epoch_loss += running_loss
        model.eval()
        count_of_correct = val(model)
        model.train()
        rate = count_of_correct / len_test
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", rate, epoch)
        if rate > best_rate[0]:
            best_rate = (rate, epoch)
            with open(f"{write_path}.pkl", "wb") as f:
                state_dict = net.state_dict()
                best_model.load_state_dict(copy.deepcopy(net.state_dict()))
                pickle.dump(
                    {
                        "net_state_dict": state_dict,
                        "current_epoch": epoch,
                        "rate": rate,
                        "optimizer": optimizer.__class__.__name__,
                        "optimizer_params": optimizer.param_groups,
                        "train_batch_size": train_batch_size,
                        "test_batch_size": test_batch_size,
                        "device": device,
                    },
                    f,
                )
        if rate < worst_rate[0]:
            worst_rate = (rate, epoch)

        print(
            f"[{epoch + 1}] rate: {rate:.4f} - {count_of_correct:3d}/{len_test}; {best_rate = }, {worst_rate = }"
        )
    print("Finished Training. Min_loss:", min_loss)
    return worst_rate, best_rate, min_loss

In [13]:
train(net, epoch_count=epoch_count)

[1,    50] loss: 0.709
[1,   100] loss: 0.716
[1,   150] loss: 0.711
[1,   200] loss: 0.707
[1,   250] loss: 0.703
[1,   300] loss: 0.700
[1,   350] loss: 0.707
[1,   400] loss: 0.693
[1,   450] loss: 0.699
[1] rate: 0.6487 - 613/945; best_rate = (tensor(0.6487), 0), worst_rate = (tensor(0.6487), 0)
[2,    50] loss: 0.701
[2,   100] loss: 0.697
[2,   150] loss: 0.700
[2,   200] loss: 0.702
[2,   250] loss: 0.695
[2,   300] loss: 0.695
[2,   350] loss: 0.692
[2,   400] loss: 0.692
[2,   450] loss: 0.697
[2] rate: 0.6540 - 618/945; best_rate = (tensor(0.6540), 1), worst_rate = (tensor(0.6487), 0)
[3,    50] loss: 0.694
[3,   100] loss: 0.696
[3,   150] loss: 0.693
[3,   200] loss: 0.693
[3,   250] loss: 0.696
[3,   300] loss: 0.695
[3,   350] loss: 0.697
[3,   400] loss: 0.692
[3,   450] loss: 0.695
[3] rate: 0.6402 - 605/945; best_rate = (tensor(0.6540), 1), worst_rate = (tensor(0.6402), 2)
[4,    50] loss: 0.695
[4,   100] loss: 0.696
[4,   150] loss: 0.698
[4,   200] loss: 0.697
[4,  

((tensor(0.3725), 218), (tensor(0.8159), 93), (0.6233018773794174, (195, 349)))